In [88]:
import requests
from bs4 import BeautifulSoup as bs
import json
import time
import os

In [94]:
b=time.time()
URL="http://www.104.com.tw/i/apis/jobsearch.cfm?fmt=8&cols=JOB,JOB_ADDR_NO_DESCRIPT,NAME,JOB_ADDRESS,JOBCAT_DESCRIPT,APPEAR_DATE,SAL_MONTH_LOW,SAL_MONTH_HIGH,LAT,LON&pgsz=2000&order=5&cat=2006001000&asc=1"
#待加入area=60010XX0XX

#市代號
bigAreaList=[len(str(i))<2 and "0"+str(i) or str(i) for i in range(1,14+1)] 
#區代號
samllAreaList=[len(str(i))<2 and "0"+str(i) or str(i) for i in range(1,29+1)] 

savedir="../data/104json"

if not os.path.exists(savedir):
    os.makedirs(savedir)
    
finishArea=[]
# finishArea=[file.split(".")[0] for file in os.listdir(savedir)] 

def get104JsonData(allpage,url):
    HRdata104=[]
    for i in range(1,allpage+1):
        res=requests.get(url+"&page=%s"%i)
        if res.status_code==200:
            alldata=json.loads(res.text)
            HRdata104+=[case for case in alldata['data'] if case["NAME"]!="104外包網"]
        res.close()
        time.sleep(1)
        print(i)
    return HRdata104

#找大區域資料
for bigkey in bigAreaList:
    area="60010%s000"%bigkey
    
    #判斷是否已有資料
    if area not in finishArea:
        URL2=URL+"&area=%s"%area
        res=requests.get(URL2)
        time.sleep(1)
        
        #判斷是否連上
        if res.status_code==200:
            data=json.loads(res.text)
            allpage=int(data["TOTALPAGE"])
            
            #判斷是否有該頁資料
            if allpage!=0:
                
                #判斷資料是否小於三千(104一次只能顯示20頁(3000筆) 多的不給抓)
                if allpage<21:
                    HRdata104=get104JsonData(allpage,URL2)
                    with open(savedir+"/"+area+".json","w") as f:
                        json.dump(HRdata104,f)
                    print(area)
                #如果資料大於三千，則抓小區域資料
                else:
                    for smallkey in samllAreaList:
                        area="60010%s0%s"%(bigkey,smallkey)
                        if area not in finishArea:
                            URL2=URL+"&area=%s"%area
                            res=requests.get(URL2)
                            time.sleep(1)
                            if res.status_code==200:
                                data=json.loads(res.text)
                                allpage=int(data["TOTALPAGE"])
                                if allpage!=0:
                                    HRdata104=get104JsonData(allpage,URL2)
                                    with open(savedir+"/"+area+".json","w") as f:
                                        json.dump(HRdata104,f)
                                    print(area)
e=time.time()
print(e-b)

1
2
3
4
5
6
6001001001
1
2
6001001002
1
2
3
4
5
6
7
8
9
6001001003
1
2
3
4
5
6
6001001004
1
2
3
4
5
6
7
8
9
10
11
6001001005
1
2
6001001006
1
2
3
4
5
6
7
8
9
6001001007
1
2
3
4
6001001008
1
2
6001001009
1
2
3
4
5
6001001010
1
2
3
6001001011
1
2
6001001012
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
6001002000
1
2
6001003000
1
2
6001004000
1
2
3
4
5
6
7
8
9
10
11
12
13
6001005000
1
2
3
4
5
6
7
8
9
6001006000
1
2
6001007000
1
6001008001
1
6001008002
1
6001008003
1
2
3
4
6001008004
1
2
6001008005
1
2
6001008006
1
2
3
4
5
6
6001008007
1
2
3
6001008008
1
6001008009
1
6001008010
1
6001008011
1
6001008012
1
6001008013
1
6001008014
1
6001008016
1
6001008017
1
6001008018
1
6001008019
1
6001008020
1
6001008021
1
6001008022
1
6001008023
1
6001008024
1
6001008025
1
6001008026
1
6001008027
1
6001008028
1
2
6001010000
1
2
6001011000
1
2
6001012000
1
2
6001013000
1
2
3
4
5
6
7
8
6001014000
372.20128870010376


In [95]:
import os
import json
savedir="../data/104json"
finishArea=[file.split(".")[0] for file in os.listdir(savedir)]
datalist=[json.load(open(savedir+"/"+file)) for file in os.listdir(savedir)]


def flat_list_v_1(the_list):
    is_nested = True
    before = the_list[:]
    while is_nested:
        is_nested = False
        now = []
        for item in before:
            if isinstance(item, list):
                now.extend(item)
                is_nested = True
            else:
                now.append(item)
        before = now
    return before

alldata=flat_list_v_1(datalist)

In [96]:
len(alldata)

26867

In [91]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()
#選擇我們想要的collection
collection = db.HRdata104
collection.drop()
collection.insert_many(alldata)

In [92]:
HRdata104=list(collection.find({}))
print("讀取完畢")
[i.pop("_id") for i in HRdata104]
import pandas as pd
df = pd.DataFrame(HRdata104)
writer = pd.ExcelWriter('../data/104pandas.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()
print("儲存完畢")

讀取完畢
儲存完畢
